In [ ]:
!pip install pymysql
!pip install pymssql

import LibData
import LibUtils
import LibML
import LibFile

import pandas as pd
import numpy as np

#jobName = '201216_Train_pilot'
jobName = '201216_Train_split1'
x, y = LibData.GetXY_s_o(jobName)

l_x, l_y = LibData.GetSplited5(x,y)


In [ ]:
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error

# 탐색 대상 함수 (XGBRegressor)
def Fit_cv(C, gamma):

    # 모델 정의
    model = SVR(kernel = 'rbf', C = C, gamma = gamma)

    # 모델 훈련 & 예측값
    avgMAE = LibML.GetMaesCV(l_x, l_y, model, yLabel)

    # return negative mae
    return -avgMAE


In [ ]:
# bayesian-optimization 라이브러리의 BayesianOptimization 클래스 import
!pip install bayesian-optimization
from bayes_opt import BayesianOptimization
import numpy as np

# pbounds = {
#     'C' : (0.01, 100),
#     'gamma' : (0.01, 0.1),
#     #'gamma' : ['auto', 'scale']    
# }

pbounds = {
    'C' : (30, 100),
    'gamma' : (0.01, 0.1),
    #'gamma' : ['auto', 'scale']    
}

# Bayesian optimization 객체 생성
# f : 탐색 대상 함수, pbounds : hyperparameter 집합
# verbose = 2 항상 출력, verbose = 1 최댓값일 때 출력, verbose = 0 출력 안함
# random_state : Bayesian Optimization 상의 랜덤성이 존재하는 부분을 통제 
yLabel = 'pv_claim'
bo=BayesianOptimization(f=Fit_cv, pbounds=pbounds, verbose=2, random_state=1)

# 메소드를 이용해 최대화 과정 수행
# init_points :  초기 Random Search 갯수
# n_iter : 반복 횟수 (몇개의 입력값-함숫값 점들을 확인할지! 많을 수록 정확한 값을 얻을 수 있다.)
# acq : Acquisition Function들 중 Expected Improvement(EI) 를 사용
# xi : exploration 강도 (기본값은 0.0)
bo.maximize(init_points=2, n_iter=30, acq='ei', xi=0.01)

# ‘iter’는 반복 회차, ‘target’은 목적 함수의 값, 나머지는 입력값을 나타냅니다. 
# 현재 회차 이전까지 조사된 함숫값들과 비교하여, 현재 회차에 최댓값이 얻어진 경우, 
# bayesian-optimization 라이브러리는 이를 자동으로 다른 색 글자로 표시하는 것을 확인할 수 있습니다

# 찾은 파라미터 값 확인
print(bo.max)

|   iter    |  target   |     C     |   gamma   |
-------------------------------------------------
|  1        | -318.1    |  41.71    |  0.07483  |
|  2        | -677.1    |  0.02144  |  0.03721  |


In [ ]:
import datetime
from sklearn.svm import SVR
import pandas as pd

def Fit2(xTrain, yTrain, xVal):
    model = SVR(kernel='rbf', C=10, gamma=0.1, epsilon=.1)
    model.fit(xTrain, yTrain)
    yVal_pred = pd.Series(model.predict(xVal), index=xVal.index)
    return model, yVal_pred

def Fit1(xTrain, yTrain, xVal):

    start = datetime.datetime.now()
    print("start time : " + start.strftime('%Y-%m-%d %H:%M:%S'))

    model_1, y_pred_1 = Fit2(xTrain, yTrain['pv_fee'], xVal)

    middle = datetime.datetime.now()
    print("dur1 : " + str(middle - start))

    model_2, y_pred_2 = Fit2(xTrain, yTrain['pv_claim'], xVal)

    end = datetime.datetime.now()
    print("dur2 : " + str(end-middle))
    print("end time : " + end.strftime('%Y-%m-%d %H:%M:%S'))
    
    y_pred = pd.concat([y_pred_1, y_pred_2], axis=1)
    y_pred.columns = ['pv_fee', 'pv_claim']

    return [model_1, model_2], y_pred, [middle - start, end-middle]


In [ ]:
import LibUtils

# 스케일링 학습1
model, xTrain_s = LibUtils.ScaleStandard(xTrain)
xVal_s = pd.DataFrame(model.transform(xVal), index=xVal.index, columns=xVal.columns)
models, y_pred, durInfo = Fit1(xTrain_s, yTrain, xVal_s)
scores = LibUtils.GetScores(yVal['pv_claim'], y_pred['pv_claim'], )
print(scores, '\n')



start time : 2020-12-20 11:21:24
dur1 : 0:15:25.950841
dur2 : 0:14:54.915996
end time : 2020-12-20 11:51:45
[423.6991550181581, 0.5059754796091639, 0.23963499636327823] 

